<a href="https://colab.research.google.com/github/ZNisaAkkilic/AI_DataScience_Projects/blob/main/fakenewsdet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Gerekli kütüphaneleri yükleyelim
!pip install pandas scikit-learn beautifulsoup4
!pip install nltk
!pip install requests


In [2]:
import pandas as pd


file_path = '/content/Fake.csv'  # Burada kendi dosya adını gir


data = pd.read_csv(file_path)


print(data.head())


                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  


In [3]:

print(data.info())

print(data.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
None
title      0
text       0
subject    0
date       0
dtype: int64


In [4]:

print(data.head())


print(data.columns)


                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  
Index(['title', 'text', 'subject', 'date'], dtype='object')


In [5]:

data = data.dropna()

print(data.shape)


(23481, 4)


In [11]:
import re
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')


stop_words = set(stopwords.words('english'))

def clean_text(text):

    text = text.lower()

    text = re.sub(r'[^\w\s]', '', text)

    text = re.sub(r'\d+', '', text)

    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

data['cleaned_text'] = data['text'].apply(clean_text)

print(data['cleaned_text'].head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    donald trump wish americans happy new year lea...
1    house intelligence committee chairman devin nu...
2    friday revealed former milwaukee sheriff david...
3    christmas day donald trump announced would bac...
4    pope francis used annual christmas day message...
Name: cleaned_text, dtype: object


In [10]:
# Etiket sütununu oluştur
# Burada sadece bir örnek kullandığımızı unutmayın, gerçek durumu yansıtmayabilir
data['label'] = data['subject'].apply(lambda x: 1 if x == 'News' else 0)

# Yeni oluşturulan etiketlerin dağılımını kontrol et
print(data['label'].value_counts())


label
0    14431
1     9050
Name: count, dtype: int64


In [12]:
print(data['label'].value_counts())


label
0    14431
1     9050
Name: count, dtype: int64


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()


X = vectorizer.fit_transform(data['cleaned_text'])


y = data['label']


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = LogisticRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[2818   58]
 [ 188 1633]]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      2876
           1       0.97      0.90      0.93      1821

    accuracy                           0.95      4697
   macro avg       0.95      0.94      0.94      4697
weighted avg       0.95      0.95      0.95      4697



In [15]:
!pip install tensorflow


In [16]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['cleaned_text'])
sequences = tokenizer.texts_to_sequences(data['cleaned_text'])


max_length = max(len(s) for s in sequences)
X_padded = pad_sequences(sequences, maxlen=max_length, padding='post')


y = data['label'].values


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_length))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_padded, y, epochs=5, batch_size=64, validation_split=0.2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
 42/294 ━━━━━━━━━━━━━━━━━━━━ 1:34:38 23s/step - accuracy: 0.5250 - loss: 0.6934